In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import numpy as np
import pretty_midi
import visual_midi
import joblib
from os.path import getsize
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

### Get data

In [23]:
bigger_examples_files = []
path = '../raw_data/pretty_midi'
directory = listdir(path)

In [24]:
# First we create a new dataset
bigger_examples_files = []
path = '../raw_data/pretty_midi'
directory = listdir(path)

for file in directory:
    if 200_000 < getsize(f'{path}/{file}') and getsize(f'{path}/{file}') < 300_000: # We don't want too big or too little files
        bigger_examples_files.append(joblib.load(f'{path}/{file}'))
    if len(bigger_examples_files) >= 10:
        break

dir = 'first_pres_dataset'
for i, file in enumerate(bigger_examples_files):
    create_nparray_dataset(file, dir, name=f'piece{i}')

In [25]:
X, y = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/piece0')
for i in range(1, 2):
    loaded = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/piece{1}')
    X = np.concatenate((X, loaded[0]))
    y = np.concatenate((y, loaded[1]))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [26]:
X.shape

(23, 128, 500)

### Make predictions

In [27]:
# X = X.reshape((X.shape[0], -1))
# y = y.reshape((y.shape[0], -1))

In [28]:
# We got to reshape data first
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# Another way to write this
y_train = y_train.reshape(
    (
        y_train.shape[0], -1
    )
)

y_test = y_test.reshape(
    (
        y_test.shape[0], -1
    )
)

In [29]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor()

#### Attempt to debug it

In [30]:
# example_file = []
# for file in directory:
#     if 200_000 > getsize(f'{path}/{file}'): # We don't want too big or too little files
#         example_file.append(joblib.load(f'{path}/{file}'))
#         break

In [31]:
# dir = 'first_pres_dataset'
# for i, file in enumerate(example_file):
#     create_nparray_dataset(file, dir, name=f'chose{i}')

In [32]:
# X_test, y_test = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/chose0')

In [33]:
# X_test, y_test = create_nparray_dataset(example_file[0], 'f', 'n', store=False)

In [34]:
# X_test = X_test.reshape((X_test.shape[0], -1))
# y_test = y_test.reshape((y_test.shape[0], -1))

#### True thing

In [35]:
# Making our predictions
predictions = tree.predict(X_test)

In [41]:
(predictions[5] == y_test[5]).mean()

0.072

In [12]:
predicted_melodies = np.array([assembled_target_to_melody(prediction) for prediction in predictions])
true_melodies = np.array([assembled_target_to_melody(melody) for melody in y_test])

In [45]:
# They should be different

In [46]:
X_test = X_test.reshape(-1, 128, 500)

In [47]:
piano_roll_to_pretty_midi(predicted_melodies[4]).write('prediction.mid')

In [48]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(predicted_melodies[4]))

Loading BokehJS ...

Column(id='2541', ...)

In [49]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(true_melodies[4]))

Loading BokehJS ...

Column(id='2740', ...)